In [138]:
import cv2

I = cv2.imread("pedestrians_empty_bg.jpg")
I.shape

(288, 352, 3)

In [139]:
import numpy as np
import matplotlib.pyplot as plt
N = 50

BUF = np . zeros (( 288 ,352 ,N) , np . uint8 )
iN = 0

cap = cv2.VideoCapture("pedestrians_input.mp4")
filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)

while(cap.isOpened()):
    ret, frame = cap.read()
    if(frame is None): 
        break
    img_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    BUF[:,:,iN] = img_gray.copy()
    iN = (iN+1)%N
    median_img = np.median(BUF,axis=2).astype(np.uint8)
    # print(median_img)
    # plt.imshow(median_img)
    # plt.show()
    abs_dif_img = ((cv2.absdiff(median_img,img_gray)>15)*255).astype(np.uint8)
    abs_dif_img = cv2.medianBlur(abs_dif_img,7)
    abs_dif_img = cv2.dilate(abs_dif_img, filter_cl1,iterations=1)
    abs_dif_img = cv2.erode(abs_dif_img, filter_cl2,iterations=1)
    
    cv2.imshow("median", abs_dif_img)
    # cv2.waitKey (1)
    cv2.imshow("frame", frame)
    cv2.waitKey (1)
cap.release()
cv2.destroyAllWindows()

In [140]:
def detect_move(I,iN,N,BUF):
    filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
    filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY)
    BUF[:,:,iN] = img_gray.copy()
    iN = (iN+1)%N
    median_img = np.median(BUF,axis=2).astype(np.uint8)
    abs_dif_img = ((cv2.absdiff(median_img,img_gray)>15)*255).astype(np.uint8)
    abs_dif_img = cv2.medianBlur(abs_dif_img,5)
    abs_dif_img = cv2.dilate(abs_dif_img, filter_cl2,iterations=3)
    abs_dif_img = cv2.erode(abs_dif_img, filter_cl2,iterations=2)
    return abs_dif_img, iN, BUF

def detect_move2(I,iN,N,BUF):
    filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
    filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY)
    BUF[:,:,iN] = img_gray.copy()
    iN = (iN+1)%N
    median_img = np.mean(BUF,axis=2).astype(np.uint8)
    abs_dif_img = ((cv2.absdiff(median_img,img_gray)>15)*255).astype(np.uint8)
    abs_dif_img = cv2.medianBlur(abs_dif_img,5)
    abs_dif_img = cv2.dilate(abs_dif_img, filter_cl1,iterations=1)
    abs_dif_img = cv2.erode(abs_dif_img, filter_cl2,iterations=1)
    return abs_dif_img, iN, BUF

In [141]:
import cv2
import numpy as np
import copy

N = 4

BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)

for i in range (301 ,1100):
    I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
    abs_dif_img, iN, BUF = detect_move2(I,iN,N,BUF)


    cv2.imshow("median", abs_dif_img)
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [142]:
import cv2
import numpy as np
import copy

N = 4

BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)

for i in range (301 ,1100):
    I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
    abs_dif_img, iN, BUF = detect_move(I,iN,N,BUF)


    cv2.imshow("median", abs_dif_img)
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [143]:
I_prev = cv2.imread (f'pedestrian/input/in{300:06d}.jpg' )
I_prev = I_prev.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

N = 4
BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>110
    I = I.astype('int')
    abs_dif_img, iN, BUF = detect_move2(I,iN,N,BUF)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.67055912804932, Recall: 0.14286701876523542, F1: 0.2355488175672394



In [144]:
I_prev = cv2.imread (f'pedestrian/input/in{300:06d}.jpg' )
I_prev = I_prev.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

N = 4
BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>110
    I = I.astype('int')
    abs_dif_img, iN, BUF = detect_move2(I,iN,N,BUF)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.67055912804932, Recall: 0.14286701876523542, F1: 0.2355488175672394



In [145]:
import cv2
import numpy as np
import copy

N = 10

BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    abs_dif_img, iN, BUF = detect_move2(I,iN,N,BUF)


    cv2.imshow("median", abs_dif_img)
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [146]:
import cv2
import numpy as np
import copy

N = 10

BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    abs_dif_img, iN, BUF = detect_move(I,iN,N,BUF)


    cv2.imshow("median", abs_dif_img)
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [147]:
I_prev = cv2.imread (f'pedestrian/input/in{300:06d}.jpg' )
I_prev = I_prev.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

N = 10
BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>110
    I = I.astype('int')
    abs_dif_img, iN, BUF = detect_move2(I,iN,N,BUF)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.5766676728784238, Recall: 0.30665255758524174, F1: 0.4003907318460815



In [148]:
I_prev = cv2.imread (f'pedestrian/input/in{300:06d}.jpg' )
I_prev = I_prev.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

N = 10
BUF = np . zeros (( 240 ,360 ,N) , np . uint8 )
iN = 0

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>110
    I = I.astype('int')
    abs_dif_img, iN, BUF = detect_move(I,iN,N,BUF)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.6762275965076606, Recall: 0.3306550169856267, F1: 0.44413925598265525



Aproksymacja sredniej i mediany (tzw. sigma-delta)

In [28]:
def sigma_delta_mean(I, BUF, alpha = 0.04):
    filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
    filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
    mean_img = BUF.astype(np.uint8)
    abs_dif_img = ((cv2.absdiff(mean_img,img_gray)>25)*255).astype(np.uint8)
    abs_dif_img = cv2.medianBlur(abs_dif_img,7)
    # abs_dif_img = cv2.dilate(abs_dif_img, filter_cl2,iterations=1)
    # abs_dif_img = cv2.erode(abs_dif_img, filter_cl2,iterations=1)
    BUF = BUF*(1-alpha) + alpha*img_gray.astype(np.float32)
    return abs_dif_img, BUF


def sigma_delta_median(I, BUF, alpha = 2):
    filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
    filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
    median_img = BUF.astype(np.uint8)
    abs_dif_img = ((cv2.absdiff(median_img,img_gray)>15)*255).astype(np.uint8)
    abs_dif_img = cv2.medianBlur(abs_dif_img,5)
    # abs_dif_img = cv2.dilate(abs_dif_img, filter_cl2,iterations=1)
    # abs_dif_img = cv2.erode(abs_dif_img, filter_cl2,iterations=1)
    BUF_plus = BUF < img_gray; 
    BUF_minus = BUF > img_gray; 
    BUF = BUF+(BUF_plus.astype(np.float32)-BUF_minus.astype(np.float32))*alpha
    return abs_dif_img, BUF
    

In [29]:
cv2.destroyAllWindows()

In [30]:
import cv2
import numpy as np
import copy

I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
I = I.astype("uint8")
img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
BUF = img_gray.astype(np.float32)


filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)

for i in range (301 ,1100):
    I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
    abs_dif_img, BUF = sigma_delta_mean(I,BUF)
    cv2.imshow("median", abs_dif_img)
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [31]:
import cv2
import numpy as np
import copy

I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
I = I.astype("uint8")
img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
BUF = img_gray.astype(np.float32)


filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)

for i in range (301 ,1100):
    I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
    abs_dif_img, BUF = sigma_delta_median(I,BUF)
    cv2.imshow("median", abs_dif_img)
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [32]:
I_prev = cv2.imread (f'pedestrian/input/in{300:06d}.jpg' )
I_prev = I_prev.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

N = 10

I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
I = I.astype("uint8")
img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
BUF = img_gray.astype(np.float32)

iN = 0

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>80
    I = I.astype('int')
    abs_dif_img, BUF = sigma_delta_mean(I,BUF)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.8721861663579998, Recall: 0.16011105815553398, F1: 0.2705551205371222



In [33]:
I_prev = cv2.imread (f'pedestrian/input/in{300:06d}.jpg' )
I_prev = I_prev.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

N = 10

I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
I = I.astype("uint8")
img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
BUF = img_gray.astype(np.float32)

iN = 0

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>80
    I = I.astype('int')
    abs_dif_img, BUF = sigma_delta_median(I,BUF)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.9197203654488287, Recall: 0.2302221728512166, F1: 0.36826191552520293



Polityka aktualizacji

In [34]:

def sigma_delta_median_cons(I, BUF, alpha = 2):
    filter_cl1 = np.array([[0,1,1,1,0],[1,1,1,1,1],[1,1,1,1,1],[1,1,1,1,1],[0,1,1,1,0]]).astype(np.uint8)
    filter_cl2 = np.array([[0,1,0],[1,1,1],[1,1,1],[1,1,1],[0,1,0]]).astype(np.uint8)
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
    median_img = BUF.astype(np.uint8)
    abs_dif_img = ((cv2.absdiff(median_img,img_gray)>20)*255).astype(np.uint8)
    abs_dif_img = cv2.medianBlur(abs_dif_img,7)
    
    
    BUF_plus = BUF < img_gray; 
    BUF_minus = BUF > img_gray; 
    BUF = BUF+((BUF_plus.astype(np.float32)-BUF_minus.astype(np.float32))*alpha)*(abs_dif_img<30).astype(np.uint8)
    return abs_dif_img, BUF
    

In [9]:
import cv2
import numpy as np
import copy

I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
I = I.astype("uint8")
img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
BUF = img_gray.astype(np.float32)
prev = np.zeros(img_gray.shape)

for i in range (301 ,1100):
    I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
    abs_dif_img, BUF = sigma_delta_median_cons(I,BUF,prev)
    prev = abs_dif_img.copy()
    cv2.imshow("median", abs_dif_img)
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [35]:
I_prev = cv2.imread (f'pedestrian/input/in{300:06d}.jpg' )
I_prev = I_prev.astype('int')

TP = 0
TN = 0
FP = 0
FN = 0

I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
I = I.astype("uint8")
img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY) 
BUF = img_gray.astype(np.float32)

iN = 0

for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>80
    I = I.astype('int')
    abs_dif_img, BUF = sigma_delta_median_cons(I,BUF)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.41161489243060245, Recall: 0.9057012314333516, F1: 0.5659994714969482



OpenCV – GMM/MOG

In [171]:
cv2.destroyAllWindows()

In [36]:
import cv2
import numpy as np
import copy

BGS_MOG2 = cv2.createBackgroundSubtractorMOG2( history = 40 , varThreshold = 24, detectShadows = True)

for i in range (301 ,1100):
    I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY)
    cv2.imshow("median", BGS_MOG2.apply(img_gray))
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [37]:

BGS_MOG2 = cv2.createBackgroundSubtractorMOG2(history = 12 , varThreshold = 2,  detectShadows = False)
TP = 0
TN = 0
FP = 0
FN = 0



for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>86
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY)

    abs_dif_img = BGS_MOG2.apply(img_gray)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.19862264930999612, Recall: 0.32885445937838864, F1: 0.2476617198481295



OpenCV – KNN

In [1]:
import cv2
import numpy as np
import copy

BGS_KNN = cv2.createBackgroundSubtractorKNN(detectShadows = True)

for i in range (301 ,1100):
    I = cv2.imread (f'pedestrian/input/in{i:06d}.jpg')
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY)
    cv2.imshow("median", BGS_KNN.apply(img_gray))
    cv2.waitKey (1)
    cv2.imshow("frame", I)
    cv2.waitKey (1)
cv2.destroyAllWindows()

In [38]:
BGS_KNN = cv2.createBackgroundSubtractorKNN(detectShadows = True)
TP = 0
TN = 0
FP = 0
FN = 0



for i in range (301 ,1100):
    I = cv2.imread (f'office/input/in{i:06d}.jpg')
    I_ground = cv2.imread (f'office/groundtruth/gt{i:06d}.png')
    I_ground = (np.sum(I_ground,axis=2)/3).astype(np.uint8)
    I_ground_b = I_ground>86
    I = I.astype("uint8")
    img_gray = cv2.cvtColor(I.astype("uint8"), cv2.COLOR_BGR2GRAY)

    abs_dif_img = BGS_KNN.apply(img_gray)
    binary_I_b = abs_dif_img>50
    TP_s = np.sum((I_ground_b & binary_I_b).astype(int))
    TN_s = np.sum((~I_ground_b & ~binary_I_b).astype(int))
    FP_s = np.sum((~I_ground_b & binary_I_b).astype(int))
    FN_s = np.sum((I_ground_b & ~binary_I_b).astype(int))
    TP+=TP_s
    TN+=TN_s
    FP+=FP_s
    FN+=FN_s
    I_prev = I

precision = TP/(TP+FP)
recall = TP/(TP+FN)
print(f"Precision: {precision}, Recall: {recall}, F1: {(2*precision*recall/(precision+recall))}\n")

Precision: 0.7956187098940253, Recall: 0.49457541432333624, F1: 0.6099755776332201

